## Notebook: Metadata usage, Metadata extraction.

This notebook illustrates exercising the metadata feature of the solution. It walks through :-
* Ingestion of a few documents with metadata
* Q & A with no metadata filtering
* Q & A with metadata filtering
* **[Extra]** Example of extracting metadata from user queries for inclusion in the RAG /generate API call.

### Install Dependencies and import required modules

In [ ]:
!pip install aiohttp
!pip install httpx

In [ ]:
import aiohttp
import httpx
import json
import base64
import os
import asyncio
import requests
from IPython.display import display, Image, Markdown

### Base Configuration

* Helper functions in the following cell
* The code assumes a docker installation of the RAG Blueprint on the same server that is running this notebook. If you have not done that, you can refer to [Get Started](../docs/deploy-docker-self-hosted.md) to start the RAG server.

In [ ]:
IPADDRESS = "localhost" # Replace this with the correct IP address if required
INGESTOR_SERVER_PORT = "8082"
INGESTOR_BASE_URL = f"http://{IPADDRESS}:{INGESTOR_SERVER_PORT}"  # Replace with your server URL if required

async def print_response(response, to_print=True):
    """Helper to print API response."""
    try:
        response_json = await response.json()
        if to_print:
            print(json.dumps(response_json, indent=2))
        return response_json
    except aiohttp.ClientResponseError:
        print(await response.text())


RAG_SERVER_PORT = "8081"
RAG_BASE_URL = f"http://{IPADDRESS}:{RAG_SERVER_PORT}"  # Replace with your server URL

rag_url = f"{RAG_BASE_URL}/v1/generate"


async def print_streaming_response_and_citations(response_generator):
    first_chunk_data = None

    async for chunk in response_generator:
        if chunk.startswith("data: "):
            chunk = chunk[len("data: "):].strip()

        if not chunk:
            continue

        try:
            data = json.loads(chunk)
        except Exception as e:
            print(f"JSON decode error: {e}")
            print(f"⚠️ Raw chunk content: {repr(chunk)}")
            continue

        choices = data.get("choices", [])
        if not choices:
            continue

        # Capture first chunk with citations (if any)
        if first_chunk_data is None and data.get("citations"):
            first_chunk_data = data

        # Stream the content
        delta = choices[0].get("delta", {})
        text = delta.get("content")
        if not text:
            message = choices[0].get("message", {})
            text = message.get("content", "")
        print(text, end='', flush=True)

    print()  # Newline after completion

    # Display citations if any
    if first_chunk_data and first_chunk_data.get("citations"):
        citations = first_chunk_data["citations"]
        for idx, citation in enumerate(citations.get("results", [])):
            doc_type = citation.get("document_type", "text")
            content = citation.get("content", "")
            doc_name = citation.get("document_name", f"Citation {idx+1}")

            display(Markdown(f"\n**Citation {idx+1}: {doc_name}**"))

            # Handle different content types properly
            if doc_type in ["image", "chart", "table"]:
                try:
                    # Try to decode as base64 and display as image
                    image_bytes = base64.b64decode(content)
                    display(Image(data=image_bytes))
                except Exception as e:
                    display(Markdown(f"⚠️ Could not decode {doc_type} content. Error: {e}"))
                    display(Markdown(f"```\nContent preview: {content[:200]}...\n```"))
            elif doc_type == "text":
                display(Markdown(f"```\n{content}\n```"))
            else:
                # Unknown content type - display as text with warning
                display(Markdown(f"⚠️ Unknown content type '{doc_type}':\n```\n{content[:500]}{'...' if len(content) > 500 else ''}\n```"))

async def generate_answer(payload):
    async with httpx.AsyncClient() as client:
        try:
            async with client.stream('POST', url=rag_url, json=payload) as response:
                async for line in response.aiter_lines():
                    yield line.strip()
        except httpx.HTTPError as e:
            print(f"Error: {e}")

### Ensure the solution is up and running 
#### Health Check Endpoint

**Purpose:**
This endpoint performs a health check on the server. It returns a 200 status code if the server is operational.

In [ ]:
async def fetch_health_status():
    """Fetch health status asynchronously."""
    url = f"{INGESTOR_BASE_URL}/v1/health"
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            try:
                resp = await response.json()
                print(resp.get("message"))
            except Exception as e:
                print("Unable to connect to ingestor server", e)
# Run the async function
await fetch_health_status()

### Create collection with the specified metadata schema

In [ ]:
COLLECTION_NAME = "cars"

In [ ]:
async def create_collection(
    collection_name=None,
    embedding_dimension: int = 2048,
    metadata_schema=None
):
    if collection_name is None:
        collection_name = []
    if metadata_schema is None:
        metadata_schema = []

    data = {
        "collection_name": collection_name,
        "embedding_dimension": embedding_dimension,
        "metadata_schema": metadata_schema
    }

    HEADERS = {"Content-Type": "application/json"}

    async with aiohttp.ClientSession() as session:
        try:
            async with session.post(f"{INGESTOR_BASE_URL}/v1/collection", json=data, headers=HEADERS) as response:
                await print_response(response)
        except aiohttp.ClientError as e:
            return 500, {"error": str(e)}

metadata_schema = [
    {
        "name": "manufacturer",
        "type": "string",
        "required": True,
        "max_length": 50,
        "description": "Car manufacturer name. Allowed Values: ford"
    },
    {
        "name": "model",
        "type": "string",
        "required": True,
        "max_length": 50,
        "description": "Car model name. Allowed Values: edge, escape"
    },
    {
        "name": "year",
        "type": "integer",
        "required": True,
        "description": "Manufacturing year. Allowed Values: 2015-2024"
    },
    {
        "name": "rating",
        "type": "float",
        "required": False,
        "description": "User rating. Allowed Values: 0.0-5.0"
    },
    {
        "name": "tags",
        "type": "array",
        "array_type": "string",
        "required": False,
        "max_length": 20,
        "description": "Document tags for categorization. Allowed Values: suv, safety, manual, technology, hybrid, eco-friendly, premium, compact, family-car, older-model, latest"
    },
    {
        "name": "created_date",
        "type": "datetime",
        "required": False,
        "description": "Document creation date. Allowed Values: Valid date format (ISO 8601 or other supported formats)"
    },
    {
        "name": "is_public",
        "type": "boolean",
        "required": False,
        "description": "Whether document is publicly accessible. Allowed Values: true, false"
    },
    {
        "name": "priority",
        "type": "integer",
        "required": False,
        "description": "Document priority level. Allowed Values: 1-10"
    },
    {
        "name": "category",
        "type": "string",
        "required": False,
        "max_length": 30,
        "description": "Document category classification. Allowed Values: owner-manual, technical-spec, validation"
    },
    {
        "name": "features",
        "type": "array",
        "array_type": "string",
        "required": False,
        "max_length": 15,
        "description": "Car features mentioned in document. Allowed Values: safety-systems, infotainment, climate-control, backup-camera, sync4, wireless-charging, adaptive-cruise, lane-keeping, blind-spot, hybrid-drivetrain, eco-mode, compact-design, fuel-efficient, electric-mode"
    },
    {
        "name": "last_updated",
        "type": "datetime",
        "required": False,
        "description": "Last update timestamp. Allowed Values: Valid date format (ISO 8601 or other supported formats)"
    },
    {
        "name": "document_type",
        "type": "string",
        "required": False,
        "max_length": 20,
        "description": "Type of document. Allowed Values: manual, specification, test"
    },
    {
        "name": "version",
        "type": "string",
        "required": False,
        "max_length": 10,
        "description": "Document version. Allowed Values: 1.0, 2.0"
    }
]

# Create the collection
await create_collection(
    collection_name=COLLECTION_NAME,
    metadata_schema=metadata_schema # Optional argument, can be commented if metadata is not to be inserted
)

### Prepare the files and metadata

* Download files
* Prepare the metadata
* Upload files into the newly created collection

In [ ]:
os.makedirs('./data', exist_ok=True)

# Mapping of filenames to source URLs
files_to_download = {
    '2024_Ford_Escape_Owners_Manual_version_1_om_EN-US.pdf': 'https://www.fordservicecontent.com/Ford_Content/Catalog/owner_information/2024_Ford_Escape_Owners_Manual_version_1_om_EN-US.pdf',
    '2023_Edge_Owners_Manual_version_2_om_EN-US.pdf': 'https://www.fordservicecontent.com/Ford_Content/Catalog/owner_information/2023_Edge_Owners_Manual_version_2_om_EN-US.pdf',
    '2015-Edge-Owner-Manual-version-2_om_EN-US_06_2015.pdf': 'https://www.fordservicecontent.com/Ford_Content/Catalog/owner_information/2015-Edge-Owner-Manual-version-2_om_EN-US_06_2015.pdf'
}

print("Downloading files ...")
for filename, url in files_to_download.items():
    destination = os.path.join('./data', filename)
    print(f"Downloading {filename}...")

    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(destination, 'wb') as f:
            for chunk in response.iter_content(1024):
                f.write(chunk)
        print(f"✅ Downloaded: {filename}")
    else:
        print(f"❌ Failed to download {filename} - HTTP {response.status_code}")

FILEPATHS = [
    "./data/2015-Edge-Owner-Manual-version-2_om_EN-US_06_2015.pdf",
    "./data/2023_Edge_Owners_Manual_version_2_om_EN-US.pdf",
    "./data/2024_Ford_Escape_Owners_Manual_version_1_om_EN-US.pdf",
    "../data/multimodal/embedded_table.pdf",
    "../data/multimodal/functional_validation.pdf"
]

CUSTOM_METADATA = [
    {
        "filename": "2015-Edge-Owner-Manual-version-2_om_EN-US_06_2015.pdf",
        "metadata": {
            "manufacturer": "ford",
            "model": "edge",
            "year": 2015,
            "rating": 4.2,
            "tags": ["suv", "safety", "manual", "older-model", "family-car"],
            "created_date": "2015-06-15 10:30:00",
            "is_public": True,
            "priority": 8,
            "category": "owner-manual",
            "features": ["safety-systems", "infotainment", "climate-control", "backup-camera"],
            "last_updated": "June 15, 2015 10:30 AM",
            "document_type": "manual",
            "version": "2.0"
        }
    },
    {
        "filename": "2023_Edge_Owners_Manual_version_2_om_EN-US.pdf",
        "metadata": {
            "manufacturer": "ford",
            "model": "edge",
            "year": 2023,
            "rating": 4.5,
            "tags": ["suv", "technology", "manual", "latest", "premium"],
            "created_date": "2023/01/20 14:15",
            "is_public": True,
            "priority": 9,
            "category": "owner-manual",
            "features": ["sync4", "wireless-charging", "adaptive-cruise", "lane-keeping", "blind-spot"],
            "last_updated": "Jan 20, 2023 2:15 PM",
            "document_type": "manual",
            "version": "2.0"
        }
    },
    {
        "filename": "2024_Ford_Escape_Owners_Manual_version_1_om_EN-US.pdf",
        "metadata": {
            "manufacturer": "ford",
            "model": "escape",
            "year": 2024,
            "rating": 4.8,
            "tags": ["compact", "hybrid", "manual", "eco-friendly", "fuel-efficient"],
            "created_date": "03/10/2024 09:45",
            "is_public": False,
            "priority": 7,
            "category": "owner-manual",
            "features": ["hybrid-drivetrain", "eco-mode", "compact-design", "fuel-efficient", "electric-mode"],
            "last_updated": "March 10, 2024",
            "document_type": "manual",
            "version": "1.0"
        }
    },
    {
        "filename": "embedded_table.pdf", # This should fail due to missing required fields
        "metadata": {
            # Missing required fields: manufacturer, model, year
            "rating": 4.0,
            "category": "technical-spec",
            "document_type": "specification"
        }
    },
    {
        "filename": "functional_validation.pdf", # This should fail due to wrong datatype
        "metadata": {
            "manufacturer": 123, # This should fail due to wrong datatype
            "model": "edge",
            "year": 2023,
            "category": "validation",
            "document_type": "test"
        }
    }
]

### Upload Documents

In [ ]:
async def upload_documents(collection_name: str = "") -> str:
    data = {
        "collection_name": collection_name,
        "blocking": False,
        "split_options": {
            "chunk_size": 512,
            "chunk_overlap": 150
        },
        "custom_metadata": CUSTOM_METADATA,
        "generate_summary": False
    }

    form_data = aiohttp.FormData()
    for file_path in FILEPATHS:
        form_data.add_field(
            "documents",
            open(file_path, "rb"),
            filename=os.path.basename(file_path),
            content_type="application/pdf"
        )

    form_data.add_field("data", json.dumps(data), content_type="application/json")

    async with aiohttp.ClientSession() as session:
        try:
            async with session.post(f"{INGESTOR_BASE_URL}/v1/documents", data=form_data) as response:
                resp_json = await response.json()
                print("Response:", resp_json)
                return resp_json.get("task_id")  # return the task_id
        except aiohttp.ClientError as e:
            print(f"Error: {e}")
            return None


async def get_task_status(
    task_id: str
):

    params = {
        "task_id": task_id,
    }

    HEADERS = {"Content-Type": "application/json"}

    async with aiohttp.ClientSession() as session:
        try:
            async with session.get(f"{INGESTOR_BASE_URL}/v1/status", params=params, headers=HEADERS) as response:
                returnval = await print_response(response, False)
                return returnval
        except aiohttp.ClientError as e:
            return 500, {"error": str(e)}

async def wait_until_task_complete(task_id):
    while True:
        result = await get_task_status(task_id=[task_id])

        if result is None:
            print("❌ No response received, retrying...")
            await asyncio.sleep(5)
            continue

        state = result.get("state", None)
        print(f"Current state: {state}")

        if state != "PENDING":
            print("✅ Task completed.")
            break

        await asyncio.sleep(15)  # wait 5 seconds before polling again
    return result

def print_ingestion_results(result):
    """Helper function to print ingestion results in a clear format"""
    print("=" * 60)
    print("📊 INGESTION RESULTS SUMMARY")
    print("=" * 60)

    # Overall status
    message = result.get("message", "No message")
    total_docs = result.get("total_documents", 0)
    print(f"📋 Status: {message}")
    print(f"🔍 Total Documents: {total_docs}")

    # Successful documents
    documents = result.get("documents", [])
    if documents:
        print(f"\n✅ SUCCESSFUL DOCUMENTS ({len(documents)}):")
        print("-" * 40)
        for doc in documents:
            doc_name = doc.get("document_name", "Unknown")
            metadata = doc.get("metadata", {})
            print(f"📄 {doc_name}")

            # Print ALL metadata fields exactly as they come
            for field_name, field_value in metadata.items():
                # Format the value properly based on type
                if isinstance(field_value, str):
                    formatted_value = f"'{field_value}'"
                elif isinstance(field_value, list):
                    formatted_value = str(field_value)  # Lists are already formatted
                elif isinstance(field_value, bool):
                    formatted_value = str(field_value).lower()  # true/false
                else:
                    formatted_value = str(field_value)

                print(f"   └─ {field_name}: {formatted_value}")
            print()

    # Failed documents
    failed_docs = result.get("failed_documents", [])
    if failed_docs:
        print(f"❌ FAILED DOCUMENTS ({len(failed_docs)}):")
        print("-" * 40)
        for doc in failed_docs:
            doc_name = doc.get("document_name", "Unknown")
            error_msg = doc.get("error_message", "Unknown error")
            print(f"📄 {doc_name}")
            print(f"   └─ Error: {error_msg}")
            print()

    # Validation errors
    validation_errors = result.get("validation_errors", [])
    if validation_errors:
        print(f"⚠️ VALIDATION ERRORS ({len(validation_errors)}):")
        print("-" * 40)
        for error in validation_errors:
            error_msg = error.get("error", "Unknown error")
            metadata = error.get("metadata", {})
            filename = metadata.get("filename", "Unknown file")
            file_metadata = metadata.get("file_metadata", {})
            print(f"📄 {filename}")
            print(f"   └─ Error: {error_msg}")
            print(f"   └─ Provided Metadata: {file_metadata}")
            print()

    print("=" * 60)

### Wait for the documents to be uploaded.

In [ ]:
task_id = await upload_documents(collection_name=COLLECTION_NAME)
print(f"Upload documents ...task_id: {task_id}")
if task_id:
    # Wait for completion and print detailed results
    result = await wait_until_task_complete(task_id)
    if result and result.get("result"):
        print_ingestion_results(result["result"])
else:
    print("⚠️ Upload failed or no task_id returned.")

### Fetch documents

Ensuring the files exist in the collection.

In [ ]:
async def fetch_documents(collection_name: str = ""):
    url = f"{INGESTOR_BASE_URL}/v1/documents"
    params = {"collection_name": collection_name}
    async with aiohttp.ClientSession() as session:
        try:
            async with session.get(url, params=params) as response:
                await print_response(response)
        except aiohttp.ClientError as e:
            print(f"Error: {e}")

await fetch_documents(collection_name=COLLECTION_NAME)

### Query: No specification of metadata etc. 

Notice that the citations included in the response include the manuals belonging to the "escape" and "edge" models.

In [ ]:
payload = {
  "messages": [
    {
      "role": "user",
      "content": "How do I reset the oil life monitor in my 2023 Ford Edge?"
    }
  ],
  "use_knowledge_base": True,
  "temperature": 0.2,
  "top_p": 0.7,
  "max_tokens": 1024,
  "reranker_top_k": 10,
  "vdb_top_k": 100,
  "vdb_endpoint": "http://milvus:19530",
  "collection_names": [COLLECTION_NAME],
  "enable_query_rewriting": True,
  "enable_reranker": True,
  "enable_citations": True,
  "model": "nvidia/llama-3.3-nemotron-super-49b-v1.5",
  "reranker_model": "nvidia/llama-3.2-nv-rerankqa-1b-v2",
  "embedding_model": "nvidia/llama-3.2-nv-embedqa-1b-v2",
  # Provide url of the model endpoints if deployed elsewhere
  # "llm_endpoint": "",
  #"embedding_endpoint": "",
  #"reranker_endpoint": "",
  "stop": [],
  "filter_expr": ''
}
await print_streaming_response_and_citations(generate_answer(payload))

### Query: Specification of simple metadata

In the previous query the responses were from multiple car models but the user really wanted the response only for their car model 
which might be the "edge"
Direct the query to a given car model i.e "edge"

Notice the citations are confined to the "edge" model.
We successfully used the metadata to limit the search to "edge" models alone. 

In [ ]:
payload = {
  "messages": [
    {
      "role": "user",
      "content": "How do I reset the oil life monitor in my 2023 Ford Edge?"
    }
  ],
  "use_knowledge_base": True,
  "temperature": 0.2,
  "top_p": 0.7,
  "max_tokens": 1024,
  "reranker_top_k": 10,
  "vdb_top_k": 100,
  "vdb_endpoint": "http://milvus:19530",
  "collection_names": [COLLECTION_NAME],
  "enable_query_rewriting": True,
  "enable_reranker": True,
  "enable_citations": True,
  "model": "nvidia/llama-3.3-nemotron-super-49b-v1.5",
  "reranker_model": "nvidia/llama-3.2-nv-rerankqa-1b-v2",
  "embedding_model": "nvidia/llama-3.2-nv-embedqa-1b-v2",
  # Provide url of the model endpoints if deployed elsewhere
  # "llm_endpoint": "",
  #"embedding_endpoint": "",
  #"reranker_endpoint": "",
  "stop": [],
  "filter_expr": 'content_metadata["model"] == "edge"'
}
await print_streaming_response_and_citations(generate_answer(payload))

Now notice, the answer generated and all citations are from the edge manuals, but they also refer to the 2015 year model despite being asked about the "2023 edge".

### Query: Specification of Compound or Complex Filter Usage - Multiple Criteria

Notice in the previous examples we used single filter conditions.
We can now combine multiple filter types in a single query for more precise results.

In the next example we use string, numeric, datetime, and array filters together with AND/OR logic.
Notice, the system handles complex combinations and returns documents matching all specified criteria.

In [ ]:
print("=" * 80)
print("🧠 COMPLEX FILTER USAGE - COMPLETE WITH LIKE")
print("=" * 80)
print("Demonstrates combining string LIKE, numeric, datetime, array, and boolean filters with AND/OR logic.\n")

payload = {
    "messages": [
        {
            "role": "user",
            "content": "What are the safety features and technology specifications for recent Ford vehicles with high ratings or Edge models from 2020-2024?"
        }
    ],
    "use_knowledge_base": True,
    "temperature": 0.2,
    "top_p": 0.7,
    "max_tokens": 1024,
    "reranker_top_k": 10,
    "vdb_top_k": 100,
    "vdb_endpoint": "http://milvus:19530",
    "collection_names": [COLLECTION_NAME],
    "enable_query_rewriting": True,
    "enable_reranker": True,
    "enable_citations": True,
    "enable_filter_generator": False,  # Disable to use manual complex filter
    "model": "nvidia/llama-3.3-nemotron-super-49b-v1.5",
    "reranker_model": "nvidia/llama-3.2-nv-rerankqa-1b-v2",
    "embedding_model": "nvidia/llama-3.2-nv-embedqa-1b-v2",
    "stop": [],
    "filter_expr": '(content_metadata["manufacturer"] like "%ford%" and content_metadata["rating"] > 4.0 and content_metadata["created_date"] between "2020-01-01" and "2024-12-31" and content_metadata["is_public"] == true) or (content_metadata["model"] like "%edge%" and content_metadata["year"] >= 2020 and content_metadata["tags"] in ["technology", "safety", "latest"] and content_metadata["rating"] >= 4.0)'
}

print("🔍 Query: What are the safety features and technology specifications for recent Ford vehicles with high ratings or Edge models from 2020-2024?")
print("🧠 Complex Filter combines:")
print("   GROUP 1 (AND logic):")
print("     • String LIKE: content_metadata['manufacturer'] like '%ford%'")
print("     • Numeric: content_metadata['rating'] > 4.0")
print("     • Datetime range: content_metadata['created_date'] between '2020-01-01' and '2024-12-31'")
print("     • Boolean: content_metadata['is_public'] == true")
print("   GROUP 2 (AND logic):")
print("     • String LIKE: content_metadata['model'] like '%edge%'")
print("     • Numeric: content_metadata['year'] >= 2020")
print("     • Array membership: content_metadata['tags'] in ['technology', 'safety', 'latest']")
print("     • Numeric: content_metadata['rating'] >= 4.0")
print("   COMBINED WITH: OR logic")
print("📋 Response:\n")

await print_streaming_response_and_citations(generate_answer(payload))

### Query: Specification of Dynamic filter expression creation

Notice in the previous examples we manually specified metadata filters using the `content_metadata` syntax.
We can now use natural language queries to automatically generate these filters.

In the next example we ask for "Edge models with high ratings above 4.0" in plain English.
Notice, the system automatically converts this to the appropriate metadata filter and returns only matching documents.

In [ ]:
# Demonstrate the new natural language filter generation capability
print("=" * 80)
print("🎯 NATURAL LANGUAGE FILTER GENERATION")
print("=" * 80)
print("This demonstrates the new AI-powered filter generation from natural language queries.")
print("The system automatically converts your question into precise metadata filters.\n")

payload = {
    "messages": [
        {
            "role": "user",
            "content": "Show me Ford vehicles with infotainment features"
        }
    ],
    "use_knowledge_base": True,
    "temperature": 0.2,
    "top_p": 0.7,
    "max_tokens": 1024,
    "reranker_top_k": 10,
    "vdb_top_k": 100,
    "vdb_endpoint": "http://milvus:19530",
    "collection_names": [COLLECTION_NAME],
    "enable_query_rewriting": True,
    "enable_reranker": True,
    "enable_citations": True,
    "enable_filter_generator": True,  # 🎯 NEW FEATURE - Enable AI filter generation
    "model": "nvidia/llama-3.3-nemotron-super-49b-v1.5",
    "reranker_model": "nvidia/llama-3.2-nv-rerankqa-1b-v2",
    "embedding_model": "nvidia/llama-3.2-nv-embedqa-1b-v2",
    "stop": [],
    "filter_expr": ""  # Will be generated automatically by AI
}

print("🔍 Query: Show me Ford vehicles with infotainment features")
print('🤖 AI will automatically generate filter: content_metadata["manufacturer"] == "ford" AND array_contains(content_metadata["features"], "infotainment")')
print("📋 Response:\n")

await print_streaming_response_and_citations(generate_answer(payload))

### Extra: Determine the metadata from the query.

It can be envisioned that the relevant metadata flags could be extracted from a user query (wherever applicable)
The below cell provides an example of how an LLM could be used to extract metadata K-V pairs that could further be used to build a filter.

In [ ]:
payload = {
  "messages": [
    {
      "role": "user",
      "content": """Extract elements from the user query if and only if they exist.
           There are two possible elements: "year" and "model".
           Return a dictionary containing only the elements found.
           Omit any keys that are not present in the query.
           All returned strings must be lowercase.
           Valid output examples:
           {"year": 2023, "model": "edge"}
           {}
           {"year": 2023}
           {"model": "edge"}
           The only allowed values are:
           For "year": 2015, 2023.
           For "model": "edge", "escape".
           User Query:
           "How do you enable and use the Rear Occupant Alert System in the 2015 escape?"
           The response should be "model": "escape", "year": 2015
        """
    }
  ],
  "use_knowledge_base": False,
  "temperature": 0.2,
  "top_p": 0.7,
  "max_tokens": 1024,
  "reranker_top_k": 2,
  "vdb_top_k": 10,
  "vdb_endpoint": "http://milvus:19530",
  "collection_names": [COLLECTION_NAME],
  "enable_query_rewriting": False,
  "enable_reranker": False,
  "enable_citations": False,
  "model": "nvidia/llama-3.3-nemotron-super-49b-v1.5",
  "reranker_model": "nvidia/llama-3.2-nv-rerankqa-1b-v2",
  "embedding_model": "nvidia/llama-3.2-nv-embedqa-1b-v2",
  # Provide url of the model endpoints if deployed elsewhere
  # "llm_endpoint": "",
  #"embedding_endpoint": "",
  #"reranker_endpoint": "",
  "stop": [],
}
extracted_metadata = await print_streaming_response_and_citations(generate_answer(payload))

Notice the response is {"model": "escape", "year": "2015"} which can then be used 
to <b>construct the query filter</b>. The LLM could easily be used to generated the "filter" itself.

### Query: Specification of Error Handling Examples

Notice in the previous examples we used valid filter expressions.
We should also understand how the system handles invalid or malformed filters.

In the next example we test various error conditions like invalid field names and unsupported operators.
Notice, the system provides clear error messages and gracefully falls back to unfiltered results when needed.

In [ ]:
print("=" * 80)
print("⚠️ ERROR HANDLING EXAMPLES")
print("=" * 80)
print("Demonstrates what happens with invalid filters and how the system handles errors.\n")

# Test 1: Invalid field name
print("🔍 Test 1: Invalid field name")
payload = {
    "messages": [
        {
            "role": "user",
            "content": "Find documents with invalid field"
        }
    ],
    "use_knowledge_base": True,
    "temperature": 0.2,
    "top_p": 0.7,
    "max_tokens": 1024,
    "reranker_top_k": 3,
    "vdb_top_k": 10,
    "vdb_endpoint": "http://milvus:19530",
    "collection_names": [COLLECTION_NAME],
    "enable_query_rewriting": True,
    "enable_reranker": True,
    "enable_citations": True,
    "enable_filter_generator": False,
    "model": "nvidia/llama-3.3-nemotron-super-49b-v1.5",
    "reranker_model": "nvidia/llama-3.2-nv-rerankqa-1b-v2",
    "embedding_model": "nvidia/llama-3.2-nv-embedqa-1b-v2",
    "stop": [],
    "filter_expr": 'content_metadata["nonexistent_field"] == "value"'  # This will cause an error
}

try:
    await print_streaming_response_and_citations(generate_answer(payload))
except Exception as e:
    print(f"❌ Expected error occurred: {str(e)}")
    print("✅ System properly handled invalid field name\n")

# Test 2: Invalid operator for field type
print("🔍 Test 2: Invalid operator for string field")
payload["filter_expr"] = 'content_metadata["manufacturer"] > 5'  # Can't use > on string

try:
    await print_streaming_response_and_citations(generate_answer(payload))
except Exception as e:
    print(f"❌ Expected error occurred: {str(e)}")
    print("✅ System properly handled invalid operator\n")

# Test 3: Invalid datetime format
print("🔍 Test 3: Invalid datetime format")
payload["filter_expr"] = 'content_metadata["created_date"] == "invalid-date"'

try:
    await print_streaming_response_and_citations(generate_answer(payload))
except Exception as e:
    print(f"❌ Expected error occurred: {str(e)}")
    print("✅ System properly handled invalid datetime format\n")

# Test 4: Empty array comparison
print("🔍 Test 4: Empty array comparison")
payload["filter_expr"] = 'content_metadata["tags"] == []'

try:
    await print_streaming_response_and_citations(generate_answer(payload))
except Exception as e:
    print(f"❌ Expected error occurred: {str(e)}")
    print("✅ System properly handled empty array comparison\n")

# Test 5: Invalid boolean value
print("🔍 Test 5: Invalid boolean value")
payload["filter_expr"] = 'content_metadata["is_public"] == "maybe"'

try:
    await print_streaming_response_and_citations(generate_answer(payload))
except Exception as e:
    print(f"❌ Expected error occurred: {str(e)}")
    print("✅ System properly handled invalid boolean value\n")

# Test 6: Mixed data types in array
print("🔍 Test 6: Mixed data types in array")
payload["filter_expr"] = 'content_metadata["tags"] in ["string", 123, true]'

try:
    await print_streaming_response_and_citations(generate_answer(payload))
except Exception as e:
    print(f"❌ Expected error occurred: {str(e)}")
    print("✅ System properly handled mixed data types in array\n")

# Test 7: Invalid syntax
print("🔍 Test 7: Invalid syntax")
payload["filter_expr"] = 'content_metadata["manufacturer"] == "ford" and'  # Incomplete expression

try:
    await print_streaming_response_and_citations(generate_answer(payload))
except Exception as e:
    print(f"❌ Expected error occurred: {str(e)}")
    print("✅ System properly handled invalid syntax\n")

# Test 8: Unsupported NULL operations
print("🔍 Test 8: Unsupported NULL operations")
payload["filter_expr"] = 'content_metadata["manufacturer"] is null'

try:
    await print_streaming_response_and_citations(generate_answer(payload))
except Exception as e:
    print(f"❌ Expected error occurred: {str(e)}")
    print("✅ System properly handled unsupported NULL operations\n")

print("=" * 80)
print("📋 ERROR HANDLING SUMMARY")
print("=" * 80)
print("✅ All error cases were properly caught and handled:")
print("   • Invalid field names")
print("   • Invalid operators for field types")
print("   • Invalid datetime formats")
print("   • Empty array comparisons")
print("   • Invalid boolean values")
print("   • Mixed data types in arrays")
print("   • Invalid syntax")
print("   • Unsupported NULL operations")
print("\n🎯 The system provides clear error messages to help users fix their filters.")

### Cleanup: Delete the collection

In [ ]:
async def delete_collections(collection_names: list[str] = ""):
    url = f"{INGESTOR_BASE_URL}/v1/collections"
    async with aiohttp.ClientSession() as session:
        try:
            async with session.delete(url, json=collection_names) as response:
                await print_response(response)
        except aiohttp.ClientError as e:
            print(f"Error: {e}")

await delete_collections(collection_names=[COLLECTION_NAME])